### LSTM creation notebook

Note: This notebook was executed in google colab since it provided easier instalation of ml packages plus the odmatrices where already grouped at a municipality level thus no privacy data was compromised.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install -U kaleido

In [ ]:
import copy
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#### Utils

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


def mean_relative_error(inv_y, inv_yhat, epsilon=1e-10):
    if len(inv_y) != len(inv_yhat):
        raise ValueError("Lists must have the same length")

    total_error = 0
    count = 0
    for y, yhat in zip(inv_y, inv_yhat):
        if y != 0:  # Only compute the relative error if the actual value is not zero
            total_error += abs(y - yhat) / (abs(y) + epsilon)
            count += 1
    mre = total_error / count if count != 0 else 0
    return mre


def plt_train_plot(history, odp):
  # plot history
  pyplot.plot(history.history['loss'], label='train')
  pyplot.plot(history.history['val_loss'], label='test')
  pyplot.legend()

  # Adding y-axis grid
  pyplot.grid(axis='y')
  # Adding titles and labels
  pyplot.title(f'Learning curves {odp} model')
  pyplot.xlabel('Epochs')
  pyplot.ylabel('Loss')
  # Saving the plot as a PNG file
  pyplot.savefig(rf"/content/drive/MyDrive/TFM/TPP/entrenamiento_{odp}.png", dpi=300, bbox_inches='tight')
  pyplot.show()


def plotly_train_plot(history, odp):
  trace_train = go.Scatter(
      x=list(range(len(history.history['loss']))),
      y=history.history['loss'],
      mode='lines',
      name='train'
  )
  trace_test = go.Scatter(
      x=list(range(len(history.history['val_loss']))),
      y=history.history['val_loss'],
      mode='lines',
      name='test'
  )
  layout = go.Layout(
      title=f'Learning curves {odp} model',
      xaxis=dict(title='Epochs'),
      yaxis=dict(title='Loss', gridcolor='lightgray'),
      plot_bgcolor='rgba(0,0,0,0)'
  )
  # Create a figure from data and layout, and plot the figure
  fig = go.Figure(data=[trace_train, trace_test], layout=layout)
  fig.update_layout(legend=dict(yanchor="top", y=0.99,xanchor="left",x=0.99),
                    title_x=0.5, title_y=0.9,)
  fig.write_image(rf"/content/drive/MyDrive/TFM/TPP/entrenamiento_{odp}.png")
  


def train_model(train_X, train_y, test_X, test_y, hidden_size):
    model = Sequential()
    # early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.add(LSTM(hidden_size, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1, activation="swish"))
    model.compile(loss='mae', optimizer='adam')
    history = model.fit(train_X, train_y, epochs=10, batch_size=32, validation_data=(test_X, test_y),
                        verbose=0, shuffle=False,)
                        # callbacks=[early_stop])
    return history.history['loss'][-1], history.history['val_loss'][-1]  # Return validation loss of the last epoch

### Gridsearch for hidden state over madrid municipality

In [ ]:
# load dataset and select just one od pair
dataset = pd.read_csv(r"/content/drive/MyDrive/TFM/TPP/final_odm_with_weather.csv", dtype="str")
dataset = dataset.astype({'trips': float, 'temperature_2m (°C)': float, 'rain (mm)': float, 'snowfall (cm)': float})
dataset["od"] = dataset["origen"]+ "|" + dataset["destino"]

# Format
dataset["fecha"] = pd.to_datetime(dataset["fecha"], format="%Y-%m-%d %H:%M:%S")
dataset = dataset.drop(columns=["origen","destino"])
# Remove first three hours and 31/10/2021
dataset = dataset[(dataset["fecha"]>datetime(2021,1,1,3)) &
                  (dataset["fecha"]<datetime(2021,10,31,4))]
# dataset = dataset.set_index("fecha")

# Make a copy to further merge
weather = dataset.drop(columns=["trips","od"]).drop_duplicates()

df = dataset[dataset.od=="28079|28079"].copy()#.drop(columns=["od"])

# Merge with weather to fill 0 trips rows
df = df[["fecha","trips"]].merge(weather, on="fecha", how="right")
df["trips"] = df["trips"].fillna(0)

# Fechas
fechas = df["fecha"].tolist()
# Sort columns
df = df[["trips", "temperature_2m (°C)", "rain (mm)","snowfall (cm)"]]
# Select values
values = df.values
# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# scaled = values
# specify the number of lag hours
n_hours = 3
n_features = 4 # ALL COLUMNS IN ORIGINAL DF
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)
print(reframed.shape)

# split into train and test sets
values = reframed.values
n_train_hours = 180 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
fechas_train = fechas[:n_train_hours]
fechas_test = fechas[n_train_hours:]
# # split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features] # Todas las filas y columnas (t-x), col var1(t)
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)
# Train_X looks like (withouth column names)
# var1(t-3)	var2(t-3)	var3(t-3)	var4(t-3)	var1(t-2)	var2(t-2)	var3(t-2) var4(t-2)	var1(t-1)	var2(t-1)	var3(t-1)	var4(t-1)
# 3	1.0	1.5	0.1	0.0	7.0	0.600000	0.1	0.0	26.0	0.600000	0.2	0.0
#
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
display(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

file_path=r"/content/drive/MyDrive/TFM/TPP/gridsearch_28079.csv"
with open(file_path,'w') as f:
  f.write("size,loss,val_loss\n")

# Optimize hidden state value
for size in tqdm([v for v in range(1,50)]):
    # Train
    train_loss, val_loss = train_model(train_X, train_y, test_X, test_y, size)
    # Save (in append so no content is loss by interruption)
    with open(file_path, "a") as file:
        file.write(f"{size},{train_loss},{val_loss}\n")

grid_search = pd.read_csv('/content/drive/MyDrive/TFM/TPP/gridsearch_28079.csv')

# Load and explore gridsearch results
grid_search = pd.read_csv('/content/drive/MyDrive/TFM/TPP/gridsearch_28079.csv')
display(grid_search.loc[grid_search.loss==grid_search.loss.min()])
display(grid_search.describe())

In [ ]:
# Plot loss vs hidden state size
fig = go.Figure()
fig.add_trace(go.Scatter(x=grid_search["size"], y=grid_search["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(x=grid_search["size"], y=grid_search["val_loss"],
                    mode='lines',
                    name='val_loss'))

fig.update_layout(#title_text='Loss and validation loss versus hidden state size', title_x=0.5, title_y=0.9,
                  xaxis_title='Hidden State Size',
                  yaxis_title='Loss Value',
                  yaxis=dict(title='Loss', gridcolor='lightgray'),
                  plot_bgcolor='rgba(0,0,0,0)')
fig.show()

#### Load, train and save each model

In [ ]:
# load dataset and select just one od pair
dataset = pd.read_csv(r"/content/drive/MyDrive/TFM/TPP/final_odm_with_weather.csv", dtype="str")
dataset = dataset.astype({'trips': float, 'temperature_2m (°C)': float, 'rain (mm)': float, 'snowfall (cm)': float})
dataset["od"] = dataset["origen"]+ "|" + dataset["destino"]

# Format
dataset["fecha"] = pd.to_datetime(dataset["fecha"], format="%Y-%m-%d %H:%M:%S")
dataset = dataset.drop(columns=["origen","destino"])
# Remove first three hours and 31/10/2021
dataset = dataset[(dataset["fecha"]>datetime(2021,1,1,3)) &
                  (dataset["fecha"]<datetime(2021,10,31,4))]
# dataset = dataset.set_index("fecha")

# Make a copy to further merge
weather = dataset.drop(columns=["trips","od"]).drop_duplicates()

# OD PAIRS TO STUDY
odps = [
    "28079|28079",
    "28074|28074",
    "28005|28005",
    "28092|28092",
    "28065|28065",
    "28058|28058",
    "28007|28007",
    "28148|28148",
    "28079|28007",
    "28079|28092",
        ]

models_dict = {}
for odp in odps:
  df = dataset[dataset.od==odp].copy()#.drop(columns=["od"])

  # Merge with weather to fill 0 trips rows
  df = df[["fecha","trips"]].merge(weather, on="fecha", how="right")
  df["trips"] = df["trips"].fillna(0)

  # Fechas
  fechas = df["fecha"].tolist()
  # Sort columns
  df = df[["trips", "temperature_2m (°C)", "rain (mm)","snowfall (cm)"]]
  # Select values
  values = df.values
  # ensure all data is float
  values = values.astype('float32')

  # ==== Train model pipeline =====
  # normalize features
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled = scaler.fit_transform(values)
  # specify the number of lag hours
  n_hours = 3
  n_features = 4 # Number of cols in original df
  # frame as supervised learning
  reframed = series_to_supervised(scaled, n_hours, 1)
  # split into train and test sets
  values = reframed.values
  n_train_hours = 180 * 24
  train = values[:n_train_hours, :]
  test = values[n_train_hours:, :]
  fechas_train = fechas[:n_train_hours]
  fechas_test = fechas[n_train_hours:]
  # # split into input and outputs
  n_obs = n_hours * n_features
  train_X, train_y = train[:, :n_obs], train[:, -n_features]
  test_X, test_y = test[:, :n_obs], test[:, -n_features]
  # reshape input to be 3D [samples, timesteps, features]
  train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
  test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

  # design network
  model = Sequential()
  model.add(LSTM(45, input_shape=(train_X.shape[1], train_X.shape[2])))
  model.add(Dense(1, activation="swish"))
  # model.add(Dense(1))
  model.compile(loss='mae', optimizer='adam')
  # fit network
  early_stop = EarlyStopping(monitor='val_loss', patience=10,
                             restore_best_weights=True)
  history = model.fit(train_X, train_y, epochs=300, batch_size=32,
                      validation_data=(test_X, test_y), verbose=2,
                      shuffle=False, callbacks=[early_stop])
###########
  # Plot learning curves
  plotly_train_plot(history, odp)
  # make a prediction
  yhat = model.predict(test_X)
  test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))
  # invert scaling for forecast
  inv_yhat = concatenate((yhat, test_X[:, -n_features+1:]), axis=1)
  inv_yhat = scaler.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,0]
  # invert scaling for actual
  test_y = test_y.reshape((len(test_y), 1))
  inv_y = concatenate((test_y, test_X[:, -n_features+1:]), axis=1)
  inv_y = scaler.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  # calculate RMSE and mre (mean relative error)
  rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
  mre = mean_relative_error(inv_y, inv_yhat)

  print(f'Test RMSE {odp}: {round(rmse,3)}')
  print(f'Test MRE {odp}: {round(mre,2)}')
  models_dict[odp] = {"model": history, "rmse_test": rmse, "mre_test": mre,
                      "#_epochs": early_stop.stopped_epoch}

  # calculate RMSE by month
  errors = {}
  for i in range(4):
    months = {0:"july",1:"august",2:"september",3:"october"}
    rmse = sqrt(mean_squared_error(inv_y[i*30*24:(i+1)*30*24],
                                   inv_yhat[i*30*24:(i+1)*30*24]))
    mre = mean_relative_error(inv_y[i*30*24:(i+1)*30*24],
                                   inv_yhat[i*30*24:(i+1)*30*24])
    errors["rmse_" + months[i]] = rmse
    errors["mre_" + months[i]] = mre
  models_dict[odp].update(errors)

  # Plots (we skipp first N-hours since they dont have predicted val)
  _d = pd.DataFrame()
  _d["Date"] = fechas_test[n_hours:]
  _d["Trips"] = inv_y
  _d["Legend"] = "Expected"
  _d2 = pd.DataFrame()
  _d2["Date"] = fechas_test[n_hours:]
  _d2["Trips"] = inv_yhat
  _d2["Legend"] = "Prediction"
  _d = pd.concat([_d, _d2], ignore_index=True)
  fig = px.line(_d, x="Date", y="Trips", color="Legend")
  fig.update_layout(height=600, width=1200, plot_bgcolor='rgba(0,0,0,0)',
                    yaxis=dict(gridcolor='lightgray'))
  fig.write_html(rf"/content/drive/MyDrive/TFM/TPP/grafico_{odp}.html")
  fig.write_image(rf"/content/drive/MyDrive/TFM/TPP/grafico_{odp}.png")


### Results exploration

1) RMSE results

In [ ]:
# Create a deep copy of the data
models_dict_copy = copy.deepcopy(models_dict)

# Remove the 'model' key from each inner dictionary in the copied data
for key in models_dict_copy:
    if 'model' in models_dict_copy[key]:
        del models_dict_copy[key]['model']

# Convert data_copy to a pandas DataFrame
models_dict_df = pd.DataFrame.from_dict(models_dict_copy, orient='index')
models_dict_df = models_dict_df.reset_index().rename(columns={"rmse_test":"rmse", "index":"OD pair"})
display(models_dict_df[["OD pair"]+[c for c in models_dict_df.columns if "rmse" in c]].round(1))

2. Training epochs per model

In [ ]:
models_dict_df.rename(columns={r"#_epochs":"total epochs"})[["OD pair", "total epochs"]]

### Aditional metadata

weather data plots

In [ ]:
weather = weather.rename(columns={"fecha":"Date"})
fig = make_subplots(rows=3, cols=1,)

fig.add_trace(
    go.Scatter(x=weather["Date"], y=weather["temperature_2m (°C)"],line=dict(color='rgb(99,110,250)')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=weather["Date"], y=weather["snowfall (cm)"],line=dict(color='rgb(99,110,250)')),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=weather["Date"], y=weather["rain (mm)"],line=dict(color='rgb(99,110,250)')),
    row=3, col=1
)
fig['layout']['yaxis']['title']='temperature (C°)'
fig['layout']['yaxis2']['title']='snowfall (cm)'
fig['layout']['yaxis3']['title']='rain (mm)'

fig.update_layout(height=600, width=1200, showlegend=False, plot_bgcolor='rgba(0, 0, 0, 0)')
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    # linecolor='black',
    gridcolor='lightgrey'
)

fig.show()